<center><img src="https://github.com/DACSS-CSSmeths/guidelines/blob/main/pics/small_logo_ccs_meths.jpg?raw=true" width="700"></center>








# Insight from your GeoDataFrame

This final session covers two main topics:

* The computation of spatial distances

* The computation of spatial indicators from the spatial data

Let me first check what I have in the map geopackage file from Brazil, already reprojected:

# II. MINING SPATIAL DATA

It is time to use  *dataPeru_indicadores.xlsx* file and the  map of Peruvian districts: *DistritosMap.zip* (zipped shape file).

Let's read the data in :

In [3]:
# !pip install openpyxl

In [4]:
# data table
import pandas as pd
import os

peruDataLink="https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/data/dataPeru_indicadores.xlsx"
datadis=pd.read_excel(peruDataLink,
                     dtype={'Ubigeo': object})
datadis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Ubigeo                    1874 non-null   object 
 1   Departamento              1874 non-null   object 
 2   Provincia                 1874 non-null   object 
 3   Distrito                  1874 non-null   object 
 4   Poblacion                 1874 non-null   int64  
 5   Superficie                1874 non-null   float64
 6   IDH2019                   1874 non-null   float64
 7   Educ_sec_comp2019_pct     1874 non-null   float64
 8   NBI2017_pct               1874 non-null   float64
 9   Viv_sin_serv_hig2017_pct  1874 non-null   float64
dtypes: float64(5), int64(1), object(4)
memory usage: 146.5+ KB


In [5]:
# map
import geopandas as gpd

peruMapaDistLink="https://github.com/CienciaDeDatosEspacial/GeoDataFrame_Analytics/raw/main/maps/DistritosMap.zip"

mapdis=gpd.read_file(peruMapaDistLink)

mapdis.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   DEPARTAMEN  1874 non-null   object  
 1   PROVINCIA   1874 non-null   object  
 2   DISTRITO    1874 non-null   object  
 3   INSTITUCIO  1874 non-null   object  
 4   geometry    1874 non-null   geometry
dtypes: geometry(1), object(4)
memory usage: 73.3+ KB


Next we will merge the indicators table into the map.

## Merging

When merging, verify the amount of resulting rows first:

In [7]:
mapdis.merge(datadis, left_on='DISTRITO', right_on='Distrito').shape

(2323, 15)

The amount of rows increases when merged. Let's do some preprocessing:

1. Capitalization:

In [8]:
# all capitals, no empty spaces before or after.

capitalizeColumns=lambda x: x.str.upper().str.strip()
datadis[['Provincia','Distrito']]=datadis[['Provincia','Distrito']].apply(capitalizeColumns)
mapdis[['PROVINCIA','DISTRITO']]=mapdis[['PROVINCIA','DISTRITO']].apply(capitalizeColumns)

2. Spanish symbols: The names may come with some symbols that may cause trouble (', ~). Let's get rid of those:

In [10]:
# !pip install unidecode

  Using cached Unidecode-1.3.8-py3-none-any.whl.metadata (13 kB)
Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)


In [11]:
import unidecode


byePunctuation=lambda x: unidecode.unidecode(x)
datadis[['Provincia','Distrito']]=datadis[['Provincia','Distrito']].map(byePunctuation)  #applymap for olderpandas
mapdis[['PROVINCIA','DISTRITO']]=mapdis[['PROVINCIA','DISTRITO']].map(byePunctuation) #applymap for olderpandas

3. Check uniqueness

In [12]:
datadis.Distrito.duplicated().sum(),mapdis.DISTRITO.duplicated().sum()

(np.int64(154), np.int64(152))

The presence of duplicates, forces we create  a column of unique values:

In [13]:
# concatenating
datadis['provDist']=["+".join(pd) for pd in zip (datadis.Provincia,datadis.Distrito)]
mapdis['provDist']=["+".join(pd) for pd in zip (mapdis.PROVINCIA,mapdis.DISTRITO)]

In [14]:
# the new column looks like this:
datadis['provDist'].head()

0    BAGUA+ARAMANGO
1       BAGUA+BAGUA
2    BAGUA+COPALLIN
3    BAGUA+EL PARCO
4       BAGUA+IMAZA
Name: provDist, dtype: object

Let's find out what is NOT matched between the  tables:

In [15]:
nomatch_df=set(datadis.provDist)- set(mapdis.provDist)
nomatch_gdf=set(mapdis.provDist)-set(datadis.provDist) 

This is the amount of rows that could not be matched:

In [16]:
len(nomatch_df), len(nomatch_gdf)

(26, 26)

Let's try renaming the districts using **fuzzy merging**:

In [19]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.6 MB/s eta 0:00:00a 0:00:01


In [20]:
# pick the closest match from nomatch_gdf for a value in nomatch_df
from thefuzz import process
[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

[('ANGARAES+HUANCA-HUANCA', ('ANGARAES+HUANCA HUANCA', 100)),
 ('ANGARAES+HUAYLLAY GRANDE', ('ANGARAES+HUALLAY GRANDE', 98)),
 ('AYMARAES+CARAYBAMBA', ('AYMARAES+CARAIBAMBA', 95)),
 ('AYMARAES+HUAYLLO', ('AYMARAES+IHUAYLLO', 97)),
 ('CHINCHEROS+ANCO_HUALLO', ('CHINCHEROS+ANCO HUALLO', 100)),
 ('HUAMANGA+SAN JOSE DE TICLLAS', ('HUAMANGA+SAN JOSE  DE TICLLAS', 98)),
 ('HUARAZ+PAMPAS', ('HUARAZ+PAMPAS GRANDE', 90)),
 ('ICA+SAN JOSE DE LOS MOLINOS', ('ICA+SAN JOSE DE  LOS MOLINOS', 98)),
 ('JAUJA+MASMA CHICCHE', ('JAUJA+MASMA-CHICCHE', 100)),
 ('JAUJA+TUNAN MARCA', ('JAUJA+TUNAN-MARCA', 100)),
 ('LEONCIO PRADO+DANIEL ALOMIAS ROBLES',
  ('LEONCIO PRADO+DANIEL ALOMIA ROBLES', 99)),
 ('LIMA+PUEBLO LIBRE', ('LIMA+MAGDALENA VIEJA', 49)),
 ('PICOTA+TRES UNIDOS', ('PICOTA+TRES-UNIDOS', 100)),
 ('PIURA+26 DE OCTUBRE', ('PIURA+VEINTISEIS DE OCTUBRE', 87)),
 ('SAN MARTIN+PAPAPLAYA', ('SAN MARTIN+PAPA-PLAYA', 98)),
 ('SECHURA+RINCONADA LLICUAR', ('SECHURA+RINCONADA-LLICUAR', 100)),
 ('TACNA+LA YARADA

If you are comfortable, you prepare a _dictionary_ of changes:

In [21]:
# is this OK?

changesDis_df={dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}
changesDis_df

{'ANGARAES+HUANCA-HUANCA': 'ANGARAES+HUANCA HUANCA',
 'ANGARAES+HUAYLLAY GRANDE': 'ANGARAES+HUALLAY GRANDE',
 'AYMARAES+CARAYBAMBA': 'AYMARAES+CARAIBAMBA',
 'AYMARAES+HUAYLLO': 'AYMARAES+IHUAYLLO',
 'CHINCHEROS+ANCO_HUALLO': 'CHINCHEROS+ANCO HUALLO',
 'HUAMANGA+SAN JOSE DE TICLLAS': 'HUAMANGA+SAN JOSE  DE TICLLAS',
 'HUARAZ+PAMPAS': 'HUARAZ+PAMPAS GRANDE',
 'ICA+SAN JOSE DE LOS MOLINOS': 'ICA+SAN JOSE DE  LOS MOLINOS',
 'JAUJA+MASMA CHICCHE': 'JAUJA+MASMA-CHICCHE',
 'JAUJA+TUNAN MARCA': 'JAUJA+TUNAN-MARCA',
 'LEONCIO PRADO+DANIEL ALOMIAS ROBLES': 'LEONCIO PRADO+DANIEL ALOMIA ROBLES',
 'LIMA+PUEBLO LIBRE': 'LIMA+MAGDALENA VIEJA',
 'PICOTA+TRES UNIDOS': 'PICOTA+TRES-UNIDOS',
 'PIURA+26 DE OCTUBRE': 'PIURA+VEINTISEIS DE OCTUBRE',
 'SAN MARTIN+PAPAPLAYA': 'SAN MARTIN+PAPA-PLAYA',
 'SECHURA+RINCONADA LLICUAR': 'SECHURA+RINCONADA-LLICUAR',
 'TACNA+LA YARADA-LOS PALOS': 'TACNA+LA YARADA LOS PALOS',
 'TARATA+ESTIQUE-PAMPA': 'TARATA+ESTIQUE PAMPA',
 'VILCAS HUAMAN+ACCOMARCA': 'VILCAS-HUAMAN+ACC

Now, make the replacements:

In [22]:
datadis.replace({'provDist':changesDis_df},inplace=True)

Now the merge can happen:

In [23]:
datadisMap=mapdis.merge(datadis, on='provDist')
# check
datadisMap.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   DEPARTAMEN                1874 non-null   object  
 1   PROVINCIA                 1874 non-null   object  
 2   DISTRITO                  1874 non-null   object  
 3   INSTITUCIO                1874 non-null   object  
 4   geometry                  1874 non-null   geometry
 5   provDist                  1874 non-null   object  
 6   Ubigeo                    1874 non-null   object  
 7   Departamento              1874 non-null   object  
 8   Provincia                 1874 non-null   object  
 9   Distrito                  1874 non-null   object  
 10  Poblacion                 1874 non-null   int64   
 11  Superficie                1874 non-null   float64 
 12  IDH2019                   1874 non-null   float64 
 13  Educ_sec_comp2019_pct     1874 non-null 

In [24]:
bye=['Departamento', 'Provincia', 'Distrito','INSTITUCIO','provDist']
datadisMap.drop(columns=bye,inplace=True)

# keeping
datadisMap.head()

,DEPARTAMEN,PROVINCIA,DISTRITO,geometry,Ubigeo,Poblacion,Superficie,IDH2019,Educ_sec_comp2019_pct,NBI2017_pct,Viv_sin_serv_hig2017_pct
0,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,"POLYGON ((-70.17413 -18.12896, -70.17461 -18.1...",230110,123662,187.74,0.578968,71.178389,15.8,0.8
1,TACNA,TACNA,POCOLLAY,"POLYGON ((-69.93475 -17.92557, -69.90467 -17.9...",230108,22319,265.65,0.645954,75.825743,16.1,0.9
2,TACNA,TACNA,CALANA,"POLYGON ((-70.11604 -17.91106, -70.11457 -17.9...",230103,3338,108.38,0.564102,77.829717,15.9,3.3
3,TACNA,TACNA,TACNA,"POLYGON ((-70.3149 -17.94498, -70.30682 -17.95...",230101,80845,1877.78,0.696613,75.491958,7.4,0.6
4,TACNA,TACNA,SAMA,"POLYGON ((-70.42497 -17.88934, -70.48022 -17.9...",230109,2679,1115.98,0.552622,70.500250,52.4,10.8


## Mining one variable

In the session on [Intro to GeoDF](https://cienciadedatosespacial.github.io/intro_geodataframe/) we did a lot on this. The main idea was simply to know the behavior of one variable, and plot it as a choropleth map.

In this case, **spatial properties** of the data were _NOT_ used at all, for example:

1) Descriptive stats would be same in a simple data frame:

In [ ]:
# statistics
datadisMap.IDH2019.describe()

2) A histogram would be same in a simple data frame:

In [ ]:
import seaborn as sea

sea.histplot(datadisMap.IDH2019, color='yellow')

3. Transform and Discretize: We also learned that we could rescale and discretize. But, given this behavior, bell-shaped,  we just need to discretize; which I will simply do using the _fisherjenks_ scheme:

In [ ]:
datadisMap.explore(
    column="IDH2019", 
    scheme="fisherjenks",
    legend=True,  
    tooltip=False, 
    popup=['DEPARTAMEN', 'PROVINCIA', 'DISTRITO'],  # show popup (on-click)
    legend_kwds=dict(colorbar=False)
)

## Spatial Properties: determining the _neighborhood_

We can compute the neighborhood for each object in a map using different options:

1. The polygons that share borders:

In [ ]:
from libpysal.weights import Queen, Rook, KNN

# rook
w_rook = Rook.from_dataframe(datadisMap,use_index=False) 

In [ ]:
w_rook.islands

2. The polygons that share at least a point:

In [ ]:
# queen
w_queen = Queen.from_dataframe(datadisMap,use_index=False)

In [ ]:
w_queen.islands

Let me show the islands detected in the previous steps:

In [ ]:
datadisMap.iloc[w_queen.islands,:].explore()

The presence of _islands_ will be problematic in more complex applications. An alternative is:

3) Nearest neighbors:

In [ ]:
# k=8 nearest neighbors
w_knn8 = KNN.from_dataframe(datadisMap, k=8)

In [ ]:
w_knn8.islands

Let's understand the differences:

In [ ]:
# first district in the GDF:
datadisMap.head(1)

In [ ]:
# amount of neighbors of that district
len(w_rook.neighbors[0])

In [ ]:
# details
datadisMap.iloc[w_rook.neighbors[0],]

In [ ]:
# see the neighbor
datadisMap.iloc[w_rook.neighbors[0] ,].plot(facecolor="yellow")

In [ ]:
# see whole area
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_rook.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
datadisMap.head(1).plot(ax=base,facecolor="red")

Let's do the same with queen neighbors:

In [ ]:
# how many
len(w_queen.neighbors[0])

In [ ]:
# details
datadisMap.iloc[w_queen.neighbors[0] ,]

In [ ]:
# see
datadisMap.iloc[w_queen.neighbors[0] ,].plot(facecolor="yellow")

In [ ]:
# whole area
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_queen.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
datadisMap.head(1).plot(ax=base,facecolor="red")

What about the _eight_ closest ones?

In [ ]:
w_knn8.neighbors[0]

In [ ]:
base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_knn8.neighbors[0],].plot(ax=base,facecolor="yellow")
datadisMap.head(1).plot(ax=base,facecolor="red")

In [ ]:
# what about k=4

w_knn4 = KNN.from_dataframe(datadisMap, k=4)

base=datadisMap[datadisMap.PROVINCIA=="TACNA"].plot()
datadisMap.iloc[w_knn4.neighbors[0],].plot(ax=base,facecolor="yellow")
datadisMap.head(1).plot(ax=base,facecolor="red")

### Exercise 6

<div class="alert-success">
        
Compute the neighbors of the capital of your country. Plot the results for each of the options.
    
</div>

## Global spatial correlation

If a spatial unit (a row) value in a variable is correlated with values of the neighbors, you know that proximity is interfering with the interpretation.

We need the neighboorhood matrix (the weight matrix) to compute spatial correlation.

In [ ]:
pd.DataFrame(*w_knn8.full()) # 1 means both are neighbors

If we standardize by row, the neighboor adds to 1:

In [ ]:
# needed for spatial correlation
w_knn8.transform = 'R'

In [ ]:
# after transformation
pd.DataFrame(*w_knn8.full()).head(12).sum(axis=1)

Spatial correlation is measured by the Moran's I statistic:

In [ ]:
from esda.moran import Moran

moranIDH = Moran(datadisMap['IDH2019'], w_knn8)
moranIDH.I,moranIDH.p_sim

A significant Moran's I suggest spatial correlation. Let's see the spatial scatter plot

In [ ]:
from splot.esda import moran_scatterplot

fig, ax = moran_scatterplot(moranIDH, aspect_equal=True)
ax.set_xlabel('IDH_std')
ax.set_ylabel('SpatialLag_IDH_std');

### Exercise 7

<div class="alert-success">
    
1. Compute the Moran's coefficient for **one** of your three  numeric variables.
    
2. Make a scatter plot for each variable.
    
</div>

## Local Spatial Correlation

We can compute a Local Index of Spatial Association (LISA -local Moran) for each map object. That will help us find spatial clusters (spots) and spatial outliers:

* A **hotSpot** is a polygon whose value in the variable is high AND is surrounded with polygons with also high values.

* A **coldSpot** is a polygon whose value in the variable is low AND is surrounded with polygons with also low values.

* A **coldOutlier** is a polygon whose value in the variable is low BUT is surrounded with polygons with  high values.

* A **hotOutlier** is a polygon whose value in the variable is high BUT is surrounded with polygons with  low values.

It is also possible that no significant correlation is detected. Let's see those values:

In [ ]:
# A LISA for each district using IDH2019
from esda.moran import Moran_Local
lisaIDH = Moran_Local(y=datadisMap['IDH2019'], w=w_knn8,seed=2022)

In [ ]:
fig, ax = moran_scatterplot(lisaIDH,p=0.05)
ax.set_xlabel('IDH_std')
ax.set_ylabel('SpatialLag_IDH_std');

You find that a district is in a **quadrant**. If the district is NOT grey, then the LISA is significant. Let's represent that information in a map:

In [ ]:
import matplotlib.pyplot as plt

from splot.esda import lisa_cluster
f, ax = plt.subplots(1, figsize=(12, 12))
plt.title('Spots and Outliers')
fig = lisa_cluster(lisaIDH, 
                   datadisMap,ax=ax,
                   legend_kwds={'loc': 'center left', 
                                'bbox_to_anchor': (0.7, 0.6)});
plt.show()

Let me add the informtion in lisaIDH to the GeoDF:

In [ ]:
# quadrant, # significance
lisaIDH.q, lisaIDH.p_sim

In [ ]:
# quadrant: 1 HH,  2 LH,  3 LL,  4 HL
pd.Series(lisaIDH.q).value_counts()

The info in **lisaIDH.q** can not be used right away, we need to add if the local spatial correlation is significant:

In [ ]:
datadisMap['IDH_quadrant']=[l if p <0.05 else 0 for l,p in zip(lisaIDH.q,lisaIDH.p_sim)  ]
datadisMap['IDH_quadrant'].value_counts()

Now, we recode:

In [ ]:
labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

datadisMap['IDH_quadrant_names']=[labels[i] for i in datadisMap['IDH_quadrant']]

datadisMap['IDH_quadrant_names'].value_counts()

In [ ]:

# custom colors
from matplotlib import colors
myColMap = colors.ListedColormap([ 'white', 'pink', 'cyan', 'azure','red'])

# Set up figure and ax
f, ax = plt.subplots(1, figsize=(12,12))
# Plot unique values choropleth including
# a legend and with no boundary lines

plt.title('Spots and Outliers')

datadisMap.plot(column='IDH_quadrant_names', 
                categorical=True,
                cmap=myColMap,
                linewidth=0.1, 
                edgecolor='k',
                legend=True,
                legend_kwds={'loc': 'center left', 
                             'bbox_to_anchor': (0.7, 0.6)},
                ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

### Exercise 8

<div class="alert-success">
    
1. Compute the Local Moran for the variables in your data that have significant spatial correlation.
    
2. Create a new column for each of those variables, with a label ('0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier').

3. Prepare a map for each of the variables analyzed, showing the spots and outliers.
    
</div>

## Mining several variables

Let me select some columns:

In [ ]:
selected_variables = ['Educ_sec_comp2019_pct',
                     'NBI2017_pct', 
                     'Viv_sin_serv_hig2017_pct']

In [ ]:
# see distribution
sea.boxplot(datadisMap[selected_variables])

Let me check their monotony:

In [ ]:
datadisMap[selected_variables].corr()

In [ ]:
sea.pairplot(
    datadisMap[selected_variables], kind="reg", diag_kind="kde"
)

Here, we can reverse the values of *Educ_sec_comp2019_pct*. First let me standardize:

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
normalized_data = scaler.fit_transform(datadisMap[selected_variables])
sea.displot(pd.melt(pd.DataFrame(normalized_data,columns=selected_variables)),
            x="value", hue="variable",kind="kde",
            log_scale=(False,False))

Let me create new variables with the standardized values:

In [ ]:
# new names
selected_variables_new_std=[s+'_std' for s in selected_variables]

# add colunms
datadisMap[selected_variables_new_std]=normalized_data

Now, it is easy to reverse:

In [ ]:
datadisMap['Educ_sec_NO_comp2019_pct_std']=-1*datadisMap.Educ_sec_comp2019_pct_std

In [ ]:
# as a result:
selected_variables_new_std = ['Educ_sec_NO_comp2019_pct_std',
                     'NBI2017_pct_std', 
                     'Viv_sin_serv_hig2017_pct_std']
sea.pairplot(
    datadisMap[selected_variables_new_std], kind="reg", diag_kind="kde"
)

### Conventional Clustering

Here, I will use the three variables to create clusters of districts. Let me explore how many clusters could be created:

In [ ]:
from scipy.cluster import hierarchy as hc


Z = hc.linkage(datadisMap[selected_variables_new_std], 'ward')
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('cases')
plt.ylabel('distance')
hc.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=1,  # font size for the x axis labels
)
plt.show()

The dendogram recommends three groups. Let me request six.

Let me use a common hierarchical technique following a agglomerative approach:

In [ ]:
from sklearn.cluster import AgglomerativeClustering as agnes

import numpy as np
np.random.seed(12345)# Set seed for reproducibility

# Initialize the algorithm, requesting 6 clusters
model = agnes(linkage="ward", n_clusters=6).fit(datadisMap[selected_variables_new_std])

# Assign labels to main data table
datadisMap["hc_ag6"] = model.labels_

In [ ]:
# see distribution of districts
datadisMap["hc_ag6"].value_counts()

We could try to find the pattern that created the clusters:

In [ ]:
datadisMap.groupby("hc_ag6")[selected_variables_new_std].mean()

Let me show you the six groups of districts which have similar behavior in three variables:

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot unique values choropleth including
# a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6", categorical=True, legend=True, linewidth=0, ax=ax
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

### Regionalization: Spatial Clustering 

Spatial clustering or Regionalization will force the contiguity of the polygons to make a cluster.

In [ ]:
# modify previous funtion call to specify cluster model with spatial constraint

model_queen = agnes(linkage="ward", 
                    n_clusters=6,
                    connectivity=w_queen.sparse).fit(datadisMap[selected_variables_new_std])
# Fit algorithm to the data
datadisMap["hc_ag6_wQueen"] = model_queen.labels_

We knew this would happen because we have islands. Then this results may not be satisfactory:

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wQueen",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

We have a couple of KNN weight matrices. Let's use those instead:

In [ ]:
model_wknn8 = agnes(linkage="ward",
                    n_clusters=6,
                    connectivity=w_knn8.sparse).fit(datadisMap[selected_variables_new_std])
datadisMap["hc_ag6_wknn8"] = model_wknn8.labels_


model_wknn4 = agnes(linkage="ward",
                    n_clusters=6,
                    connectivity=w_knn4.sparse).fit(datadisMap[selected_variables_new_std])
datadisMap["hc_ag6_wknn4"] = model_wknn4.labels_

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(10, 12))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wknn8",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
# Set up figure and ax
f, ax = plt.subplots(1, figsize=(10, 12))
# Plot unique values choropleth including a legend and with no boundary lines
datadisMap.plot(
    column="hc_ag6_wknn4",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

We could evaluate two aspects of these clustering results:

* “Compactness” of cluster shape, using the isoperimetric quotient (IPQ). This compares the area of the region to the area of a circle with the same perimeter as the region. For this measure, more compact shapes have an IPQ closer to 1, whereas very elongated or spindly shapes will have IPQs closer to zero. For the clustering solutions, we would expect the IPQ to be very small indeed, since the perimeter of a cluster/region gets smaller the more boundaries that members share.

In [ ]:
from esda import shape as shapestats
results={}
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the region polygons using a dissolve
    regions = datadisMap[[cluster_type, "geometry"]].to_crs(24892).dissolve(by=cluster_type)
    # compute the actual isoperimetric quotient for these regions
    ipqs = shapestats.isoperimetric_quotient(regions)
    # cast to a dataframe
    result = {cluster_type:ipqs}
    results.update(result)
# stack the series together along columns
pd.DataFrame(results)

An alternative could be _convex_hull_ratio_, simply the division of the area of the cluster by the area of its convex hull.

In [ ]:
from esda import shape as shapestats
results={}
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the region polygons using a dissolve
    regions = datadisMap[[cluster_type, "geometry"]].to_crs(24892).dissolve(by=cluster_type)
    # compute the actual convex hull quotient for these regions
    chullr = shapestats.convex_hull_ratio(regions)
    # cast to a dataframe
    result = {cluster_type:chullr}
    results.update(result)
# stack the series together along columns
pd.DataFrame(results)

In both cases, the non spatial clusters do better.

* Goodness of fit. Here we have two metrics:
    - metrics.calinski_harabasz_score
    - silhouette_score

In [ ]:
from sklearn import metrics

fit_scores = []
for cluster_type in ("hc_ag6_wknn4", "hc_ag6_wknn8", "hc_ag6"):
    # compute the CH score
    ch_score = metrics.calinski_harabasz_score(
        # using scaled variables
        datadisMap[selected_variables_new_std],
        # using these labels
        datadisMap[cluster_type],
    )
    sil_score = metrics.silhouette_score(
        # using scaled variables
        datadisMap[selected_variables_new_std],
        # using these labels
        datadisMap[cluster_type],
    )
    # and append the cluster type with the CH score
    fit_scores.append((cluster_type, ch_score,sil_score))


# re-arrange the scores into a dataframe for display
pd.DataFrame(
    fit_scores, columns=["cluster type", "CH score", "SIL score"]
).set_index("cluster type")

Again, the conventional clustering beats the others, as you want bigger values in both.

### Exercise 9

<div class="alert-success">
    
Use your three variables to carry out the cluster/regional analysis.
    
</div>


### Conventional Regression



In [ ]:
from pysal.model import spreg

dep_var_name=['NBI2017_pct']
ind_vars_names=['Educ_sec_comp2019_pct','Viv_sin_serv_hig2017_pct']


ols_model = spreg.OLS(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    spat_diag = True,
    moran=True,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names)

print(ols_model.summary)

### Spatial Regression

* Spatial Lag Regression

In [ ]:
moranNBI = Moran(datadisMap[dep_var_name], w_knn8)
moranNBI.I,moranNBI.p_sim

In [ ]:
fig, ax = moran_scatterplot(moranNBI, aspect_equal=True)
ax.set_xlabel('NBI')
ax.set_ylabel('SpatialLag_NBI');

In [ ]:
lag_model = spreg.ML_Lag(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names
    )

print(lag_model.summary)

* Spatial Error Regression

In [ ]:

moranError = Moran(ols_model.u, w_knn8)
moranError.I,moranError.p_sim

In [ ]:
fig, ax = moran_scatterplot(moranError, aspect_equal=True)
ax.set_xlabel('OlsError')
ax.set_ylabel('SpatialOlsError');

In [ ]:
err_model = spreg.ML_Error(
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names
    )

print(err_model.summary)

* Spatial Error Regression, correcting heteroscedasticy.

In [ ]:
error_Het_model = spreg.GM_Error_Het(    
    # Dependent variable
    datadisMap[dep_var_name].values,
    # Independent variables
    datadisMap[ind_vars_names].values,
    # Spatial weights matrix
    w=w_knn8,
    # Dependent variable name
    name_y=dep_var_name[0],
    # Independent variable name
    name_x=ind_vars_names,
)
print(error_Het_model.summary)

### Exercise 10

<div class="alert-success">
    
Use your three variables to carry out regression analysis (conventional and spatial).
    
</div>